In [32]:
%load_ext autoreload
%autoreload

import os
import pandas as pd
import numpy as np
import asyncio
#from functools import partial
import twitterscraper as tws
from IPython.core.debugger import set_trace
from tqdm import tqdm_notebook
# from konlpy.tag import Okt
import texcrapy
import json
from dask import compute, delayed

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
df = pd.read_excel('keywords and logos.xlsx', sheet_name='20190215')[['shortname','kw_supporter','kw_supported','keywords']]; df
_or = lambda kw: ' OR '.join(['#' + k.strip() for k in kw.split(',')])

qry_base = {row.shortname:_or(row.keywords) for row in df.itertuples()}
supporters = df.shortname[df.kw_supporter==True]
qry_sup = ' OR '.join([qry_base[sup] for sup in supporters]); qry_sup
qry = {row.shortname: '(' + qry_base[row.shortname] + ') AND (' + qry_sup + ')' if row.kw_supported==True else qry_base[row.shortname] for row in df.itertuples()}

In [31]:
qry

{'ootd': '#dailylook OR #outfit OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 OR #아웃핏',
 'category': '#designer OR #streetwear OR #luxury OR #casual OR #sportswear OR #디자이너 OR #길거리패션 OR #럭셔리 OR #캐주얼 OR #캐쥬얼 OR #스포츠웨어 OR #스포츠패션',
 'fashion': '#fashion OR #brand OR #fashionbrand OR #shoe OR #shoes OR #clothes OR #wear OR #apparel OR #패션 OR #브랜드 OR #패션브랜드 OR #옷 OR #의류 OR #슈즈 OR #신발',
 '8seconds': '#8seconds OR #8second OR #에잇세컨즈 OR #8세컨즈',
 'adidas': '#adidas OR #아디다스',
 'adidasoriginal': '#adidasoriginal OR #아디다스오리지널스 OR #아디다스오리지널 OR #아디다스오리지날스 OR #아디다스오리지날',
 'asics': '#asics OR #아식스',
 'balenciaga': '#balenciaga OR #balensiaga OR #발렌시아가',
 'bally': '#bally OR #발리',
 'beanpole': '#beanpole OR #빈폴',
 'benetton': '#benetton OR #베네통',
 'blackyak': '#blackyak OR #블랙야크',
 'buckaroo': '#buckaroo OR #버커루',
 'burberry': '#burberry OR #버버리',
 'calvinklein': '#calvinklein OR #케빈클라인 OR #켈빈클라인 OR #캘빈클라인',
 'canadagoose': '#canadagoose OR #캐나다구스',
 'chanel':

In [2]:
df = pd.read_excel('keywords.xlsx', skiprows=1)[['item','keyword_eng','keyword_kr','intersection']].set_index('item')
_or = lambda kw: ' OR '.join(['#' + k.strip() for k in kw.split(',')])

df['keyword'] = df.keyword_eng + ', ' + df.keyword_kr
df['qry'] = df.keyword.apply(_or)

intersect_with = ['ootd', 'category', 'fashion']
screener = ' OR '.join(df.loc[intersect_with, 'qry'])
df.loc[df.intersection==True, 'qry'] = '(' + df.qry + ') AND (' + screener + ')'

qry = dict(df.qry)

In [3]:
qry

{'ootd': '#dailylook OR #outfit OR #selfie OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디',
 'category': '#designer OR #streetwear OR #luxury OR #casual OR #sportswear OR #디자이너 OR #길거리패션 OR #럭셔리 OR #캐주얼 OR #캐쥬얼 OR #스포츠웨어 OR #스포츠패션',
 'fashion': '#fashion OR #brand OR #fashionbrand OR #shoe OR #shoes OR #clothes OR #wear OR #apparel OR #패션 OR #브랜드 OR #패션브랜드 OR #옷 OR #의류 OR #슈즈 OR #신발',
 '8seconds': '#8seconds OR #에잇세컨즈',
 'adidas': '#adidas OR #아디다스',
 'adidasoriginal': '#adidasoriginal OR #아디다스오리지널스 OR #아디다스오리지널 OR #아디다스오리지날스 OR #아디다스오리지날',
 'asics': '#asics OR #아식스',
 'balensiaga': '#balensiaga OR #발렌시아가',
 'bally': '#bally OR #발리',
 'beanpole': '#beanpole OR #빈폴',
 'benetton': '#benetton OR #베네통',
 'blackyak': '#blackyak OR #블랙야크',
 'buckaroo': '#buckaroo OR #버커루',
 'burberry': '#burberry OR #버버리',
 'calvinklein': '#calvinklein OR #케빈클라인 OR #켈빈클라인 OR #캘빈클라인',
 'canadagoose': '#canadagoose OR #캐나다구스',
 'chanel': '#chanel OR #샤넬',
 'coach': '#co

In [6]:
# 요게 필요한가?
with open('twitter_data/qry.json', 'w') as f:
    json.dump(qry, f)

In [15]:
%%time
what = ['id', 'timestamp', 'text']
texcrapy.scrap(qry, what=what, lang='ko', end='2019-01-31', download_to='twitter_data')

INFO: queries: ['#dailylook OR #outfit OR #selfie OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 since:2010-01-01 until:2010-06-15', '#dailylook OR #outfit OR #selfie OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 since:2010-06-15 until:2010-11-28', '#dailylook OR #outfit OR #selfie OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 since:2010-11-28 until:2011-05-13', '#dailylook OR #outfit OR #selfie OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 since:2011-05-13 until:2011-10-26', '#dailylook OR #outfit OR #selfie OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 since:2011-10-26 until:2012-04-09', '#dailylook OR #outfit OR #selfie OR #bestofstreetwear OR #beststreetoutfit OR #ootd OR #outfitoftheday OR #데일리룩 OR #오오티디 since:2012-04-09 until:2012-09-22', '#dailylook OR #outfit OR #selfie OR

Wall time: 3h 9min 20s


In [12]:
pd.read_pickle('twitter_data/aldo (2010.01.01-2019.01.31).pkl')

aldo                                                     \
                      id                                               text   
0     424094692545150976  http://ift.tt/1hluftb  sunny!!!오늘뱅쿠버날씨짱따듯..♡♡ ...   
1     369020028038811649  Blog Post: 2013-08-18 04H, Korea Topic : <a hr...   
2     369003690138423296  Blog Post: 2013-08-18 03H, Korea Topic : <a hr...   
3     363875224820781056  이제 드디어 UFC 페더급 극강의 챔피언 #Aldo 와 #KoreanZombie #...   
4     363865180851408896  #코리안좀비 #koreanzombie #정찬성 선수 파이팅 #chan sung ju...   
5     935086430786871296  오늘처럼 쇼핑많이 해보긴 \n태어나서 처음 :D  \n\n#H&M\n#ZARA\n#...   
6     933522192549195781  .\n.\n미세먼지가 불만이었던 날 \n부츠 좋아보인다는 말에 기분 풀림\n-\n스...   
7     909785203387768835  언제 먹어도 맛있는 알도 육포.\n걸으면서도 질겅질겅 \n육포는 사랑입니다.\n맥주...   
8     908325568982257665  오늘도 테이크아웃, 나의 아지트:-) #울산맛집 #달동맛집 #알도 #알도피자 @알도...   
9     907234403084062720  수고한 이세미씨와 가볍게 한잔먹고 후딱 집에가도, 살찌겠지. \n.\n.\n#울산맛...   
10    906512460156329984  맥주+ 알도표 #쉬림프스캠피 내내 노래를 부르다 오늘 못참고 테이크아웃으로 집에서 ...   
11    905074229070233600  #울산맛집 #달동맛집 #알도 우리골목만세! 이제 무조건 여기. 맛보장 @알도 htt...   
12    894410224655769600  알도, 개성과 사랑의 메시지 담은 ''I &#10084;... '' 캠페인 전개 h...   
13    873152390865735680  #심한꼭지노출 #레걸들망가 #성인영화세상 #아가씨들포토 #미시들유방 #aldo ★최...   
14    871558904076877824  알도 울린 할로웨이, 다섯쌍둥이도 삼키나     \n#알도 #할로웨이 #다섯쌍둥이 ...   
15    871233488350978049  [UFC] 할로웨이, 알도에 3R TKO 승리..페더급 챔피언 등극 #알도 #할로웨...   
16    871233337947332608  [UFC] 할로웨이, 알도에 3R TKO 승리..페더급 챔피언 등극 #조제 #알도 ...   
17    862280846501335041  #심한꼭지노출 #레걸들망가 #성인영화세상 #아가씨들포토 #미시들유방 #aldo ★최...   
18    858711140137713665  .\n마음에 들었던 오늘 #스타일 \n마지막은 편안한 #알도 #자넬라\n오늘 사진 ...   
19    858208077228777472  벌써 샌들의 계절!  매력 넘치는 #핑크 #알도 #자렐라 와 흰바지의 조화는 쵝오\...   
20    851348678216359936  #심한꼭지노출 #레걸들망가 #성인영화세상 #아가씨들포토 #미시들유방 #aldo ★최...   
21   1029967999669657600  #알도 #ALDO #패션 #신발 #슈즈 #18fw #2018fw #fw #신상 #신...   
22   1016754866251677696  #알도 땅가림 천약 http://www.iij.kr/760554558  http:/...   
23    480341446751289345  [Facebook update] 140621 가로수길 #ALDO 오픈행사해요! 저도...   
24    477203073966673920  #강남 http://ift.tt/SE4k81  http://ift.tt/1q8Ylq...   
25    476051667125682176  #알도 #aldo #알도매장 #강남역 #앤트윤 #antyoon #beautifulo...   
26    473398446322249728  칼퇴+택시로 크록스, 알도 다 와봤지만 찾고있는 신발은 모두 품절 중... ㅠㅠ #...   
27    458017584474882048  "@adamlambert: #Aldo Lamberti -- my Italian co...   
28    444365352588173312  알도VS라마스 알도 스탠딩 전략 동영상 http://goo.gl/fb/uKTd5  ...   
29    444264867030380544  #강남역 #강남역 #알도 #ALDO 우오아 http://ift.tt/1ko9SNE ...   
..                   ...                                                ...   
141   602824069747068929  주말~월욜까지 #매드맥스 #스파이 #악의연대기 러쉬!! 끄악 다 좋음. 앞으로 이렇...   
142   602773653550342144  #스파이 멜리사 막 욕 쏟아 붇다가 뒤돌아서면 걱정하면서 쪼는 거 귀염귀염 ㅋㅋㅋㅋ...   
143   602481872430444544  #주드로 오빠가 좀 많이 잘생기게 나온다고 하는 #스파이 봤는데, 영화가 너무 웃겨...   
144   602469159163695104  #스파이 엄마도 엄청 웃어네요ㅋ 나도 많이 뿜음! #알도 멋있다고 했더니 걱정스럽다...   
145   602464133938749441  #스파이 보고 나옴. 나오자마자 외침 #알도 가 제일 웃기닭 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 말투...   
146   602411273788067841  #스파이 멜리사 막 욕 쏟아 붇다가 뒤돌아서면 걱정하면서 쪼는 거 귀염귀염 ㅋㅋㅋㅋ...   
147   602119471461859328  #주드로 오빠가 좀 많이 잘생기게 나온다고 하는 #스파이 봤는데, 영화가 너무 웃겨...   
148   602106782001934336  #스파이 엄마도 엄청 웃어네요ㅋ 나도 많이 뿜음! #알도 멋있다고 했더니 걱정스럽다...   
149   602101768537780225  동생이랑 #스파이 보고 나옴. 나오자마자 외침 #알도 가 제일 웃기닭 ㅋㅋㅋㅋㅋㅋㅋ...   
150   602048879698128896  #스파이 멜리사 막 욕 쏟아 붇다가 뒤돌아서면 걱정하면서 쪼는 거 귀염귀염 ㅋㅋㅋㅋ...   
151   601287806648877056  #스파이 수잔(#멜리사) 낸시(#미란다) 레이나(#로즈) 버릴 캐릭이 없음 + 마성...   
152   601151564032712704  [위드뉴욕] #어스타일포유 #구하라 패션 #알도 샌들。상품문의는 위드뉴욕 http:...   
153   594699011761827842  [위드뉴욕] #소녀시대 #공항패션 #서현 착용 #알도 샌들。 pic.twitter....   
154   588951581254623232  #알도 #aldo #세일 #sale #슈즈 #이벤트 #event\n\nHappy S...   
155   583160774371221504  청청하게_\n\n#aldoreview #aldo #aldogram #알도 #gill...   
156   580288778067156992  #알도 #알도그램 #슈퍼핑크 #벚꽃 #카모 #후르트링 #핑크 #aldo #aldog...   
157   578728300056604672  #ALDO #알도 #알도그램 #이태원 #매장오픈 #리뉴얼 #이벤트 #럭키백 #ins...   
158   576038878286598145  판매액 전액 에이즈퇴치에 기부되는 알도의 착한팔찌.\n남자끼리 대동단결.\n\n